In [1]:
import pandas as pd

save_path = '/home/rsn-dl/deep-learning/Dasom/result/'
df = pd.read_json(save_path+'sampling_data_코로나_5.json')

In [8]:
import torch
PATH = "/home/rsn-dl/deep-learning/Dasom/klue-roberta-ftned.pt"
model = torch.load(PATH)
model.eval()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [2]:
target_df = df

In [3]:
emb_shape = len(target_df['embedding'][0])

In [4]:
import numpy as np
db_array = np.array(target_df['embedding'].to_list(), dtype=np.float32)

In [5]:
import faiss

index = faiss.IndexIDMap(faiss.IndexFlatIP(emb_shape))
index.add_with_ids(db_array, np.array(range(0, len(target_df))))

In [6]:
faiss.write_index(index, 'corona_twitter') #인덱스 쓰기

In [7]:
index = faiss.read_index('corona_twitter') #역직렬화

In [13]:
import time
def search(query):
   t=time.time()
   query_vector = model.encode([query])
   k = 5
   top_k = index.search(query_vector, k)
   print('totaltime: {}'.format(time.time()-t))
   return [target_df['title'][_id] for _id in top_k[1].tolist()[0]]

In [17]:
query=str(input())
results=search(query)
print('results :')
for result in results:
   print('\t',result)

단계적 일상회복이 이제는 시작됬군!
totaltime: 0.02735447883605957
results :
	 ‘단계적 일상회복’ 위드코로나!
일상회복을 위한 첫 걸음으로
음식점, 카페 등 영업제한 해제?

자세한 내용 가치보다로 같이보자!

#가치보다 #KTV국민방송 #위드코로나 #백신패스
#사적모임 #회식… https://t.co/zBdzHSnpF3
	 위드코로나 영업제한 푸는 걸 11월 1일 오전 6시부터 했어야....
	 금일부터 00~06시 영업제한 해제 되었습니다.
코로나 2단계에 따라 방역수칙 준수 부탁드립니다.
	 진짜 거리두기, 영업제한..이제는 지친다..코로나 전에 굶어죽겠다..
	 @kyunghyang "어제 코로나19 신규 확진자 363명...이틀째 300명대"
확산세가 안전정적 추세인데...
이제 '영업제한시간'을 해제하고 일상으로 돌아가 경제에 전념해야 한다. 
한편 나라가 디비… https://t.co/2pPxSYSct0


In [19]:
query=str(input())
results=search(query)
print('results :')
for result in results:
   print('\t',result)

아마 메타버스 관련 글은 없을껄?
totaltime: 0.012840509414672852
results :
	 행정가 이재명 “코로나19, 영업제한 소상공인 보상법 반드시 제정해야”(정치/사회) https://t.co/vMN2PKBPkU 

전투형 노무현 이재명!!  again 2002
	 '코로나 비관' 관장 사망? 헬스장 아닌 재활체육센터
#오마이뉴스
https://t.co/FBz2UqLbYh
이런 기데기 새끼들!!!!
나도 오늘까지 관장이 영업제한때문에 자살한 걸로 알고 있었는데;;;
대… https://t.co/mSeGapG9YS
	 코로나 버티려면 영업제한, 금지업종에 집중지원해야 한다고 생각한다. . 초기에는 이재명처럼 하지만 지금은 이재명처럼 하면 언발에 오줌누기밖에 되지 않는다고 생각된다. .
	 합격하고 난 뒤엔 법학 선행 공부도 했다. 코로나 영업제한 제외하고 늘 새벽 2시까지 독서실에서 공부했다.
	 코로나 9시 영업제한 이전에
새벽에 가도 항상 택시아재들 옹기종기 모여서 국밥 한그릇하는게 눈에 보였는데
이제는 아예 보이질않음ᆢ


In [20]:
class FaissKMeans:
    def __init__(self, n_clusters=10, n_init=100, max_iter=300):
        self.n_clusters = n_clusters
        self.n_init = n_init
        self.max_iter = max_iter
        self.kmeans = None
        self.cluster_centers_ = None
        self.inertia_ = None

    def fit(self, X, y):
        self.kmeans = faiss.Kmeans(d=X.shape[1],
                                   k=self.n_clusters,
                                   niter=self.max_iter,
                                   nredo=self.n_init)
        self.kmeans.train(X.astype(np.float32))
        self.cluster_centers_ = self.kmeans.centroids
        self.inertia_ = self.kmeans.obj[-1]

    def predict(self, X):
        return self.kmeans.index.search(X.astype(np.float32), 1)[1]

In [21]:
D = emb_shape
K = 5  # The number of clusters

# Setup
kmeans = faiss.Kmeans(d=D, k=K, niter=20, verbose=True)
# For GPU(s), run the following line. This will use all GPUs
# kmeans = faiss.Kmeans(d=D, k=K, niter=20, verbose=True, gpu=True)

# Run clustering
kmeans.train(db_array)

# Error for each iteration
print(kmeans.obj)  # array with 20 elements

# Centroids after clustering
print(kmeans.centroids.shape)  # (10, 128)

# The assignment for each vector.
dists, ids = kmeans.index.search(db_array, 1)  # Need to run NN search again
print(ids.shape)

# Params
print("D:", kmeans.d)
print("K:", kmeans.k)
print("niter:", kmeans.cp.niter)

Clustering 851 points in 768D to 5 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
[78087.390625   47895.8046875  46711.16796875 46279.55859375lance=1.469 nsplit=0         
 46101.53515625 45957.0859375  45838.71484375 45758.08203125
 45722.34765625 45707.01953125 45689.734375   45671.2421875
 45661.6640625  45655.1015625  45650.328125   45647.1015625
 45645.25390625 45644.96484375 45644.96484375 45644.96484375]
(5, 768)
(851, 1)
D: 768
K: 5
niter: 20

